<a href="https://colab.research.google.com/github/nmalby/Daily-Transaction-Audit-Extractor/blob/main/Daily-Transaction-Audit-Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tabula-py

In [2]:
import io
import re
import tabula
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from google.colab import files

In [ ]:
uploaded = files.upload()

In [4]:
sectionCodes = {
    'supervis 1': 205,
    'coordina 1': 206,
    'pockets': 207,
    'backs': 208,
    'bodies': 209,
    'slv/col/fly': 211,
    'fronts': 212,
    'parts': 213,
    'coordina 2': 221,
    'shell prep': 222,
    'therm/mb 1': 223,
    'therm/mb 2': 224,
    'coordina 3': 236,
    'assembly': 237,
    'finishing': 238,
    'hardware': 239,
    'misc lbr': 266,
    'training': 270,
    'engineer': 295,
    'cutting indirect': 305,
    'cutting direct': 310}

remove_pass_one =   {'Transaction Type':['----------------','Earnings Inquiry','IPR...Labor Code:','Lunch Start','Lunch Stop','Sign-OFF','difications.......', 'Operator Inquiry', 'Bundle Inquiry','Bulk Start','Bundle Quit'],
                     'Problem':['Bun already done','Currently on job','Job not found', 'Not signed on', 'Cur on lunch','Cur on other W/S','Already signd on']}
remove_pass_two =   {'Trans':['for Emp',"999999"]}

areaBounds = {
    1: {'header':[95,85,105,120], 'body':[110, 35, 565, 750]},
    2: {'header':[85,85,100,120], 'body':[100, 35, 555, 750]} }

columnsBounds = {'time': 70, 'trans': 110, 'trans type': 205, 'problem': 300, 'ipr': 320, 'job': 348, 'code': 400, 'nbr': 430, 'rate fctr': 495, 'sect': 520, 'number': 560, 'qty': 610}

In [5]:
def pdfToExcel(pdf, pageRange):
    ID_header = tabula.read_pdf(pdf, pages = pageRange, area=areaBounds[2]['header'])
    df_raw = tabula.read_pdf(pdf, pages = pageRange, area=areaBounds[2]['body'], columns=list(columnsBounds.values()), guess=False)
    df_final = pd.DataFrame()

    date = re.search(r'([0-9]+(\.[0-9]+)+)', pdf.name).group().replace(".", "/")

    for page, ID in zip(df_raw, ID_header):
        page = page[~page.isin({'Transaction Type':['----------------']}).any(axis='columns')]
        page.drop(columns=['Code Pay', 'Rate  Fctr', 'Sect', 'Sp  Qty', 'SAHAmountEFF'], inplace=True)

        page['Nbr'].fillna(value=next(iter(ID)), inplace=True)
        page['Job'].fillna(method='ffill', inplace=True)

        pCriteria = (page['Transaction Type'] == 'Bundle Start')
        nCriteria = (page['Problem'] != 'Bun already done') & (page['Problem'] != 'Bundle not found') & (page['Problem'] != 'Cur on lunch') &(page['Problem'] != 'Not signed on') #& (page['Problem'] != 'Bun Done - other')
        page = page.loc[pCriteria].loc[nCriteria]

        page.reset_index(drop=True, inplace=True)

        df_final = pd.concat([df_final, page], ignore_index=True)

    df_final.rename(columns={'IPR': 'Date', 'Nbr': 'ID'}, inplace=True)
    
    df_final['Job'].fillna(method='ffill', inplace=True)
    df_final['Date'].fillna(value=date, inplace=True)

    # display(df_final)
    df_final.to_excel(pdf.name.split('/')[-1].split('.p')[0] + '.xlsx')


In [ ]:
for pdf_raw in uploaded:
    pdf = io.BytesIO(uploaded[pdf_raw])
    pdf.name = pdf_raw

    pdfToExcel(pdf, "2-500")